In [ ]:
pip install nbstripout


In [ ]:

!pip install pathway==0.5.2 bokeh --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires sqlglot>=23.6.3, but you have sqlglot 10.6.1 which is incompatible.
ibis-framework 9.5.0 requires sqlglot<25.21,>=23.4, but you have sqlglot 10.6.1 which is incompat

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("dataset.csv")

# Combine datetime columns
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                  format="%d-%m-%Y %H:%M:%S")

# Map vehicle and traffic weights
vehicle_weights = {"bike": 0.5, "car": 1.0, "truck": 1.5}
df["VehicleTypeWeight"] = df["VehicleType"].map(vehicle_weights)
traffic_weights = {"low": 0.3, "medium": 0.6, "high": 1.0}
df["TrafficWeight"] = df["TrafficConditionNearby"].map(traffic_weights)

# Sort values and reset index
df = df.sort_values("Timestamp").reset_index(drop=True)

# Select and clean data for stream
stream_df = df[[
    "SystemCodeNumber", "Timestamp", "Occupancy", "Capacity",
    "QueueLength", "IsSpecialDay", "VehicleTypeWeight", "TrafficWeight"
]].copy()

# Replace blank strings with NaN
stream_df.replace("", pd.NA, inplace=True)

# Clean: force numeric conversion and fill NaNs
for col in ["QueueLength", "IsSpecialDay", "VehicleTypeWeight", "TrafficWeight"]:
    stream_df[col] = pd.to_numeric(stream_df[col], errors="coerce").fillna(0.0)

# Also convert Occupancy and Capacity to numeric safely
stream_df["Occupancy"] = pd.to_numeric(stream_df["Occupancy"], errors="coerce").fillna(0).astype(int)
stream_df["Capacity"] = pd.to_numeric(stream_df["Capacity"], errors="coerce").fillna(1).astype(int)

# ✅ Save cleaned CSV
stream_df.to_csv("parking_stream.csv", index=False)


In [ ]:
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)




In [ ]:

import pathway as pw

class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    IsSpecialDay: int
    VehicleTypeWeight: float
    TrafficWeight: float

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)


In [ ]:

# Demand-based pricing model
demand = (
    (data.Occupancy / data.Capacity) * 1.0 +
    data.QueueLength * 0.8 +
    data.IsSpecialDay * 0.6 +
    data.VehicleTypeWeight * 0.7 -
    data.TrafficWeight * 0.9
)

normalized_demand = demand / 10
base_price = 10.0
dynamic_price = base_price * (1 + 0.8 * normalized_demand)

priced_stream = data.select(
    SystemCodeNumber=data.SystemCodeNumber,
    Timestamp=data.Timestamp,
    Occupancy=data.Occupancy,
    Capacity=data.Capacity,
    DynamicPrice=dynamic_price
)


In [ ]:

# Save final pricing output
pw.io.csv.write(
    table=priced_stream,
    filename="/tmp/parking_priced.csv"
)



In [ ]:
pw.run()


Output()

In [ ]:
import pandas as pd

df_result = pd.read_csv("/tmp/parking_priced.csv")
df_result.head()


,SystemCodeNumber,Timestamp,Occupancy,Capacity,DynamicPrice,time,diff
0,Others-CCCPS105a,2016-10-05 10:30:00,1339,2009,13.653201,1751867617830,1
1,BHMNCPNST01,2016-10-04 12:29:00,379,485,15.025155,1751867617830,1
2,Shopping,2016-10-05 09:30:00,1138,1920,12.738167,1751867617830,1
3,Others-CCCPS119a,2016-10-04 14:57:00,529,2803,12.990981,1751867617830,1
4,BHMMBMMBX01,2016-10-04 16:31:00,471,687,12.172472,1751867617830,1


from matplotlib import pyplot as plt
_df_0['Occupancy'].plot(kind='hist', bins=20, title='Occupancy')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Capacity'].plot(kind='hist', bins=20, title='Capacity')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['DynamicPrice'].plot(kind='hist', bins=20, title='DynamicPrice')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('SystemCodeNumber').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_16['Timestamp'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_16, x='Occupancy', y='Timestamp', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_17['SystemCodeNumber'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_17, x='Capacity', y='SystemCodeNumber', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_18['Timestamp'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_18, x='Capacity', y='Timestamp', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [44]:
from google.colab import drive
import shutil

# Mount Google Drive to access your file
drive.mount('/content/drive')

# Copy your file from Google Drive into Colab local environment
shutil.copy('/content/drive/MyDrive/Dynamic_Pricing2.ipynb', '/content/Dynamic_Pricing2.ipynb')


Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Dynamic_Pricing2.ipynb'

In [45]:
!ls "/content/drive/MyDrive"

 08401042023
'ChainFlow AI.gdoc'
 Classroom
'Colab Notebooks'
 conclusion.gdoc
'Data collection and dataset.gdoc'
 Finlit.drawio
'Google AI Studio'
 grastech.jpg
 ideaflux.gdoc
 IMG-20241230-WA0133.jpg
 Mindmap.drawio
'NOC (1).pdf'
 NOC.gdoc
 NOC.pdf
'oops file.gdoc'
'Photos vids'
 Picsart_24-12-02_17-17-04-038.jpg
'Prototype (1).pdf'
 Prototype.gdoc
 Prototype.pdf
'resume 1.pdf'
 Resume.gdoc
 Screenshot_20250223_170323_WinZO.jpg
 Screenshot_20250223_170528_Chrome.jpg
'Screenshot_20250223_170831_Google Play Store.jpg'
 Screenshot_20250321_082553_Flitpay.jpg
 Screenshot_20250321_082944_Chrome.jpg
 Screenshot_20250321_083326_Chrome.jpg
 sdi.gdoc
 Snigdha
'Snigdha (1)'
'Snigdha Choudhury '
 Snigdha-ChoudhuryResume.pdf
'snigdha IGDTUW'
'Summary of my work in the quizmaker app.gdoc'
'Summary of My Work on QuizMaker.gdoc'
'TOM ASSIGNMENT -1.pptx'
'Untitled document (1).gdoc'
'Untitled document (2).gdoc'
'Untitled document.gdoc'


In [46]:
!ls "/content/drive/MyDrive/Colab Notebooks"


'Copy of zomato_analysis.ipynb'   dynamic_pricing.ipynb   zomato_analysis.ipynb
 Dynamic_Pricing2.ipynb		  Submission.ipynb
